Proposed reforms include:

1. Introduce a transitional period before disenrolling PIP recipients.
2. Increase the threshold for Pension Credit.
3. Repeal the two-child limit.

| Policy reform                       |   2025 |   2026 |   2027 |   2028 |   2029 |   2025-29 |
|:-----------------------------|-------:|-------:|-------:|-------:|-------:|----------:|
| PIP reform with 6 months TP  |   -3   |    0   |    0   |    0   |    0   |      -3   |
| PIP reform with 12 months TP |   -5.5 |    0   |    0   |    0   |    0   |      -5.5 |
| PIP reform with 18 months TP |   -5.5 |   -3   |    0   |    0   |    0   |      -8.6 |
| Raise PC threshold by £1,000 |   -1.6 |   -1.6 |   -1.6 |   -1.5 |   -1.4 |      -7.8 |
| Raise PC threshold by £2,000 |   -3.2 |   -3.3 |   -3.2 |   -3.2 |   -3.1 |     -16   |
| Raise PC threshold by £3,000 |   -4.9 |   -4.9 |   -4.9 |   -4.8 |   -4.7 |     -24.1 |
| Repeal two-child limit       |   -1.5 |   -1.6 |   -1.7 |   -2   |   -2.1 |      -8.9 |

In [1]:
from policyengine import Simulation
import pandas as pd
import plotly.express as px
from policyengine.utils.charts import *


def get_threshold_change_reform(
    threshold_change: float,
):
    return {
        "gov.dwp.pension_credit.guarantee_credit.minimum_guarantee.SINGLE": 221.85 + threshold_change / 52,
        "gov.dwp.pension_credit.guarantee_credit.minimum_guarantee.COUPLE": 338.61 + threshold_change / 52,
    }

raise_pc_threshold_1k = get_threshold_change_reform(1000)
raise_pc_threshold_2k = get_threshold_change_reform(2000)
raise_pc_threshold_3k = get_threshold_change_reform(3000)

two_child_limit_repeal = {
    "gov.dwp.universal_credit.elements.child.limit.child_count": 99,
    "gov.dwp.tax_credits.child_tax_credit.limit.child_count": 99,
}



/Users/nikhilwoodruff/policyengine/policyengine-app/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def pip_reform(
    transitional_protection_months: int = 0,
) -> Simulation:
    sim = Simulation(
        country="uk",
        scope="macro",
    )
    baseline = sim.baseline_simulation
    
    has_enhanced_level = (baseline.calculate("pip_dl_category").values == "ENHANCED")

    data = baseline.to_input_dataframe()

    SPLIT = 0.7 # SPLIT% of claimants are in the treatment group

    import pandas as pd

    data_copy_b = data.copy()
    data_copy_r = data.copy()

    for col in data_copy_r.columns:
        if "_id" in col:
            data_copy_r[col] = data_copy_r[col].apply(lambda x: 1e9 + x)
        if "_weight" in col:
            data_copy_r[col] = data_copy_r[col] * SPLIT

    for col in data_copy_b.columns:
        if "_weight" in col:
            data_copy_b[col] = data_copy_b[col] * (1 - SPLIT)

    combined_data = pd.concat([data_copy_b, data_copy_r])

    sim.baseline_simulation = Simulation(
        country="uk",
        scope="macro",
        data=combined_data,
    ).baseline_simulation
    sim.reform_simulation = Simulation(
        country="uk",
        scope="macro",
        data=combined_data,
    ).baseline_simulation

    import numpy as np

    for year in range(2025, 2030):
        sim.reform_simulation.set_input(
            "pip_dl",
            year,
            list(baseline.calculate("pip_dl", year).values) + list(baseline.calculate("pip_dl", year).values * np.where(
                has_enhanced_level, 1, max(min((transitional_protection_months / 12) - (year - 2025), 1), 0)
            ))
        )
    return sim

simulations = [
    Simulation(
        country="uk",
        scope="macro",
        reform=reform
    ) for reform in [
        raise_pc_threshold_1k,
        raise_pc_threshold_2k,
        raise_pc_threshold_3k,
        two_child_limit_repeal,
    ]
]

for transitional_protection_months in [0, 6, 12, 18]:
    simulations.append(
        pip_reform(transitional_protection_months)
    )

INFO:root:Using Google Cloud Storage for download.
INFO:policyengine.utils.data.caching_google_storage_client:Syncing policyengine-uk-data-private, enhanced_frs_2022_23.h5, 1.11.5 to cache
INFO:policyengine.utils.data.simplified_google_storage_client:Downloading policyengine-uk-data-private, enhanced_frs_2022_23.h5
INFO:policyengine.utils.data.caching_google_storage_client:Copying downloaded data for policyengine-uk-data-private, enhanced_frs_2022_23.h5 to enhanced_frs_2022_23.h5
INFO:root:Using Google Cloud Storage for download.
INFO:policyengine.utils.data.caching_google_storage_client:Syncing policyengine-uk-data-private, enhanced_frs_2022_23.h5, 1.11.5 to cache
INFO:policyengine.utils.data.caching_google_storage_client:Cache exists and crc is unchanged for policyengine-uk-data-private, enhanced_frs_2022_23.h5.
INFO:policyengine.utils.data.caching_google_storage_client:Copying downloaded data for policyengine-uk-data-private, enhanced_frs_2022_23.h5 to enhanced_frs_2022_23.h5
INFO:r

In [3]:
reform_labels = [
    "Raise PC threshold by £1,000",
    "Raise PC threshold by £2,000",
    "Raise PC threshold by £3,000",
    "Repeal two-child limit",
    "PIP reform with no TP",
    "PIP reform with 6 months TP",
    "PIP reform with 12 months TP",
    "PIP reform with 18 months TP",
]

In [4]:
budgetary_impacts = []
years = []

for year in range(2025, 2030):
    print(year)
    for sim in simulations:
        budgetary_impacts.append(sim.reform_simulation.calculate("gov_balance", year).sum()/1e9 - sim.baseline_simulation.calculate("gov_balance", year).sum()/1e9)
        years.append(year)


2025
2026
2027
2028
2029


In [5]:
import pandas as pd

df = pd.DataFrame({
    "reform": reform_labels * 5,
    "year": years,
    "budgetary_impact": budgetary_impacts,
})

In [6]:
# cols = year, rows=reform, values=budgetary_impact
df_pivot = df.pivot_table(
    index="reform",
    columns="year",
    values="budgetary_impact",
    aggfunc="sum"
).reset_index()

In [7]:
df_pivot.round(1)

year,reform,2025,2026,2027,2028,2029
0,PIP reform with 12 months TP,0.0,5.6,5.7,5.9,6.0
1,PIP reform with 18 months TP,0.0,2.6,5.7,5.9,6.0
2,PIP reform with 6 months TP,2.6,5.6,5.7,5.9,6.0
3,PIP reform with no TP,5.5,5.6,5.7,5.9,6.0
4,"Raise PC threshold by £1,000",-1.6,-1.5,-1.4,-1.4,-1.3
5,"Raise PC threshold by £2,000",-3.1,-3.1,-3.0,-2.9,-2.8
6,"Raise PC threshold by £3,000",-4.7,-4.7,-4.6,-4.5,-4.4
7,Repeal two-child limit,-1.6,-1.7,-1.8,-2.1,-2.2


In [8]:
df_pivot[:2][df_pivot.columns[1:]] = df_pivot.loc[:2][df_pivot.columns[1:]] - df_pivot.loc[2][df_pivot.columns[1:]]

In [9]:
df_pivot[2025][1]

0.0

In [10]:
for year in range(2025, 2030):
    for i in range(3):
        df_pivot[year][i] = df_pivot[year][i] - df_pivot[year][3]

In [11]:
df_pivot.drop(index=3, inplace=True)

In [12]:
df_pivot["2025-29"] = df_pivot[[2025, 2026, 2027, 2028, 2029]].sum(axis=1)

In [13]:
print(df_pivot.round(1).to_markdown(index=False))

| reform                       |   2025 |   2026 |   2027 |   2028 |   2029 |   2025-29 |
|:-----------------------------|-------:|-------:|-------:|-------:|-------:|----------:|
| PIP reform with 12 months TP |   -5.5 |    0   |    0   |    0   |    0   |      -5.5 |
| PIP reform with 18 months TP |   -5.5 |   -3   |    0   |    0   |    0   |      -8.5 |
| PIP reform with 6 months TP  |   -2.9 |    0   |    0   |    0   |    0   |      -2.9 |
| Raise PC threshold by £1,000 |   -1.6 |   -1.5 |   -1.4 |   -1.4 |   -1.3 |      -7.2 |
| Raise PC threshold by £2,000 |   -3.1 |   -3.1 |   -3   |   -2.9 |   -2.8 |     -15   |
| Raise PC threshold by £3,000 |   -4.7 |   -4.7 |   -4.6 |   -4.5 |   -4.4 |     -22.8 |
| Repeal two-child limit       |   -1.6 |   -1.7 |   -1.8 |   -2.1 |   -2.2 |      -9.4 |
